<a href="https://colab.research.google.com/github/KorStats/skku/blob/main/(%EC%B5%9C%EC%A2%852)_kobert_nsmc2_aug_eda_replace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 필요 라이브러리 설치

In [1]:
!pip install transformers datasets accelerate scikit-learn optuna sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install transformers

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, classification_report
from sklearn.preprocessing import LabelEncoder
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import re
import platform
import transformers
import joblib

In [4]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


# 80% 데이터셋

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   NSMC2_0.5_eda_swap.pkl	 NSMC2_0.8_eda_swap.pkl   이전
'My Drive'	    NSMC2_0.8_aug_eda_swap.pkl	'실험 결과 정리.xlsx'	  졸업논문


In [6]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.8_aug_eda_replace_final.csv', encoding='utf-8-sig')

In [7]:
df['label'].value_counts()

,count
label,
1,5000
0,5000


In [8]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [9]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

In [10]:
def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)

In [11]:
train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 12416.73it/s]


In [12]:
# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

In [13]:
# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

tokenization_kobert.py:   0%|          | 0.00/10.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/monologg/kobert:
- tokenization_kobert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


tokenizer_78b3253a26.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

In [14]:
# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

In [15]:
# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [16]:
# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [18]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

In [19]:
training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

Epoch,Training Loss,Validation Loss,F1
1,0.305400,0.417331,0.828096
2,0.218900,0.417564,0.839210
3,0.203700,0.505354,0.822895
4,0.111900,0.627048,0.829005
5,0.072700,0.695036,0.818240
6,0.031800,0.704679,0.837101
7,0.056000,0.785932,0.833720
8,0.021700,0.814947,0.839383
9,0.010800,0.840787,0.834702
10,0.014300,0.858011,0.837415


Elapsed Time: 1052.05 seconds
Average CPU Usage: 17.25%


In [22]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.8_aug_eda_replace.pkl')

#print("모델이 저장되었습니다.")

In [23]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.8_eda_replace.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

In [24]:
test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]
selected_columns7.head(10)

,text,category_prediction
0,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...,1
2,폴리스스토리 시리즈는 뉴까지 버릴께 하나도 없음 최고,0
3,와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지,1
4,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화,1
5,사랑을 해본사람이라면 처음부터 끝까지 웃을수 있는영화,1
6,완전 감동입니다 다시봐도 감동,1
7,개들의 나오나요 나오면 보고 싶음,1
8,내 나이와 같은 영화를 지금 본 나는 감동적이다하지만 훗날 다시보면대사하나하나 그 ...,1
9,영화의 오페라화라고 해야할 작품 극단적 평갈림은 어쩔 수 없는 듯,0


In [25]:
a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [26]:
selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.8_eda_replace_prediction.csv', encoding='utf-8-sig')

In [27]:
from sklearn.metrics import f1_score
import pandas as pd
df=pd.read_csv('/content/nsmc2_0.8_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8881    0.7780    0.8294      1000
           1     0.8025    0.9020    0.8493      1000

    accuracy                         0.8400      2000
   macro avg     0.8453    0.8400    0.8394      2000
weighted avg     0.8453    0.8400    0.8394      2000



# 50% 데이터셋

In [28]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.5_aug_eda_replace_final.csv', encoding='utf-8-sig')

In [29]:
df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [30]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 13053.25it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [32]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.249700,0.452238,0.797755
2,0.181900,0.478308,0.792037
3,0.096300,0.629672,0.813055
4,0.054100,0.871382,0.780927
5,0.036100,0.833395,0.837887
6,0.021100,0.896068,0.825886
7,0.006200,0.891338,0.832313
8,0.013000,0.934573,0.818463
9,0.014600,1.076035,0.813092
10,0.025100,1.034071,0.821001


Elapsed Time: 1059.36 seconds
Average CPU Usage: 16.05%


In [33]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.5_eda_replace.pkl')

#print("모델이 저장되었습니다.")

In [34]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.5_eda_replace.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.5_eda_replace_prediction.csv', encoding='utf-8-sig')



In [35]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.5_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.8860    0.7770    0.8279      1000
           1     0.8014    0.9000    0.8479      1000

    accuracy                         0.8385      2000
   macro avg     0.8437    0.8385    0.8379      2000
weighted avg     0.8437    0.8385    0.8379      2000



# 10% 데이터셋

In [36]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.1_aug_eda_replace_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [37]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)


Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 13354.19it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

In [39]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.165100,0.926528,0.469566
2,0.147400,0.877908,0.570130
3,0.083600,1.179781,0.585515
4,0.036600,1.085536,0.678516
5,0.023200,1.960569,0.516302
6,0.023200,1.522148,0.626624
7,0.014400,1.927887,0.594675
8,0.020800,2.069234,0.572297
9,0.001800,1.964709,0.597666
10,0.020400,2.000726,0.592993


Elapsed Time: 1055.52 seconds
Average CPU Usage: 15.20%


In [40]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.1_eda_replace.pkl')

#print("모델이 저장되었습니다.")

In [41]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.1_eda_replace.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.1_eda_replace_prediction.csv', encoding='utf-8-sig')



In [42]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.1_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9203    0.4390    0.5944      1000
           1     0.6316    0.9620    0.7626      1000

    accuracy                         0.7005      2000
   macro avg     0.7760    0.7005    0.6785      2000
weighted avg     0.7760    0.7005    0.6785      2000



# 2% 데이터셋

In [43]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/졸업논문/NSMC2_0.02_aug_eda_replace_final.csv', encoding='utf-8-sig')

df_test=pd.read_excel('/content/gdrive/MyDrive/졸업논문/DATA/NSMC_test.xlsx')

In [44]:
df['label'].value_counts()

,count
label,
1,5000
0,5000


In [45]:
data_list = []
for q, label in zip(df['document'], df['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

#from sklearn.model_selection import train_test_split
#dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0, shuffle=True)

train=pd.DataFrame(data_list)

data_list = []
for q, label in zip(df_test['document'], df_test['label'])  :
    data = []
    data.append(q)
    data.append(str(label))
    data_list.append(data)

test=pd.DataFrame(data_list)

train.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)
test.rename(columns = {0 : 'text', 1 : 'category'}, inplace = True)

def clean_korean_documents(documents):
    # 텍스트 정제 (특수기호만 제거)
    for i, document in tqdm(enumerate(documents), total=len(documents), desc="Cleaning Special Characters"):
        document = re.sub(r'[^\w\sㄱ-ㅣ가-힣]', '', document)  # 특수기호만 제거, 정규 표현식
        documents[i] = document
    return documents

def remove_words_with_numbers(text):
    return ' '.join([word for word in text.split(',') if not re.search(r'\d', word)])

# 한글이 아닌 문자를 제거하는 함수
def remove_non_korean(text):
    return re.sub(r'[^ㄱ-ㅣ가-힣\s]', '', text)


train['text'] = clean_korean_documents(train['text'])
train['text'] = train['text'].apply(lambda x: ','.join(x.split()))
train['text'] = train['text'].apply(remove_words_with_numbers)
train['text'] = train['text'].apply(remove_non_korean)

test['text'] = clean_korean_documents(test['text'])
test['text'] = test['text'].apply(lambda x: ','.join(x.split()))
test['text'] = test['text'].apply(remove_words_with_numbers)
test['text'] = test['text'].apply(remove_non_korean)

# 3. 학습 데이터와 테스트 데이터에서 필요한 열 추출 및 라벨 인코딩
X_train = train['text']
y_train = train['category']
X_test = test['text']
y_test=test['category']

# 4. 라벨 인코딩
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded=label_encoder.fit_transform(y_test)
num_labels = len(label_encoder.classes_)

# 5. KoBERT 모델을 위한 토크나이저 및 데이터셋 생성
model_name = 'monologg/kobert'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# 텍스트를 토큰화하는 함수 정의
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

# 데이터셋 생성 및 토크나이즈
train_df = pd.DataFrame({'text': X_train, 'label': y_train_encoded})
test_df = pd.DataFrame({'text': X_test, 'label': y_test_encoded})
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# PyTorch 텐서 형식으로 변환
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# 5. KoBERT 모델 설정 및 학습 파라미터 최적화
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels).to(device)

#CPU 측정

import psutil
import time
from transformers import Trainer, TrainingArguments

# CPU 사용량 및 속도 측정 함수
def measure_cpu_usage_and_time(train_function):
    start_time = time.time()  # 시작 시간 기록
    start_cpu = psutil.cpu_percent(interval=None)  # 시작 CPU 사용량

    train_function()  # 학습 함수 호출

    end_time = time.time()  # 종료 시간 기록
    end_cpu = psutil.cpu_percent(interval=None)  # 종료 CPU 사용량

    elapsed_time = end_time - start_time
    avg_cpu_usage = (start_cpu + end_cpu) / 2

    print(f"Elapsed Time: {elapsed_time:.2f} seconds")
    print(f"Average CPU Usage: {avg_cpu_usage:.2f}%")

Cleaning Special Characters: 100%|██████████| 2000/2000 [00:00<00:00, 11941.05it/s]


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
import optuna
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score
import numpy as np

# `compute_metrics` 함수
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    f1 = f1_score(labels, predictions, average='macro')  # F1 점수 계산
    return {'eval_f1': f1}  # 'eval_f1' 반환

# Optuna를 활용한 하이퍼파라미터 튜닝
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

training_args = TrainingArguments(
    output_dir='/content',
    num_train_epochs=10,  # 에포크 수 고정
    per_device_train_batch_size=64,
    learning_rate=5.0e-05,  # 최적화된 학습률
    weight_decay=0.01,  # 최적화된 weight decay
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='/content',
    logging_steps=10,
    fp16=True,
    optim="adamw_torch",  # AdamW fused 적용 (PyTorch 2.0 이상)
    disable_tqdm=False,
    load_best_model_at_end=True,
    metric_for_best_model="f1",  # 가장 좋은 모델을 accuracy 기준으로 선택
    greater_is_better=True,  # accuracy가 높은 모델을 저장
    save_total_limit=2,  # 저장할 모델 개수 제한
    report_to="none"
)

# 최적의 파라미터로 Trainer 구성
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

trainer7 = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,  # 전체 학습 데이터로 평가
    compute_metrics=compute_metrics
)

# CPU 사용량 및 속도 측정
measure_cpu_usage_and_time(trainer7.train)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,F1
1,0.041000,1.289262,0.336433
2,0.053700,1.883258,0.342952
3,0.060600,1.940503,0.335103
4,0.054500,2.078202,0.333111
5,0.052900,2.220996,0.350153
6,0.009600,2.311349,0.344117
7,0.023800,2.259085,0.379912
8,0.016800,2.138944,0.433268
9,0.010600,2.519047,0.420633
10,0.006900,2.889438,0.396630


Elapsed Time: 1054.66 seconds
Average CPU Usage: 15.35%


In [47]:
# 모델 저장
#joblib.dump(trainer7, '/content/gdrive/MyDrive/NSMC2_0.02_eda_replace.pkl')

#print("모델이 저장되었습니다.")

In [48]:
# 8. 저장된 모델로 테스트 데이터 예측 및 결과 저장
# 저장된 모델 불러오기

#loaded_model = joblib.load('/content/gdrive/MyDrive/NSMC2_0.02_eda_replace.pkl')

# 불러온 모델을 사용하여 예측 수행
predictions7 = trainer7.predict(test_dataset)

test_predictions7 = np.argmax(predictions7.predictions, axis=1)
decoded_test_predictions7 = label_encoder.inverse_transform(test_predictions7)

test["category_prediction"] = decoded_test_predictions7

selected_columns7 = test[['text', 'category_prediction']]

a=[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

selected_columns7['y_true']=pd.DataFrame(a)
selected_columns7 = selected_columns7.rename(columns={"category_prediction": "y_pred"})
selected_columns7.to_csv('/content/nsmc2_0.02_eda_replace_prediction.csv', encoding='utf-8-sig')



In [49]:
from sklearn.metrics import f1_score
import pandas as pd

df=pd.read_csv('/content/nsmc2_0.02_eda_replace_prediction.csv', encoding='utf-8-sig')

from sklearn.metrics import classification_report, f1_score

# y_true와 y_pred를 추출
y_true = df['y_true']
y_pred = df['y_pred']

# classification_report를 사용하여 각 클래스별 F1-score 출력
report = classification_report(y_true, y_pred, digits=4)
print(report)

              precision    recall  f1-score   support

           0     0.9259    0.1000    0.1805      1000
           1     0.5243    0.9920    0.6860      1000

    accuracy                         0.5460      2000
   macro avg     0.7251    0.5460    0.4333      2000
weighted avg     0.7251    0.5460    0.4333      2000

